<a href="https://colab.research.google.com/github/mangrue/IDD-Resources/blob/main/IDD_Datasets_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setting up environment**

In [ ]:
# Install specific libraries
#! pip install torch
#! pip install transformers
#! pip install pycaret
#! pip install pandas
#! pip install numpy
#! pip install pycaret
#! pip install matplotlib
#! pip install -U scikit-learn
#! pip install transformers==2.8.0
#!pip install --upgrade huggingface_hub
!pip install evaluate -q
!pip install datasets -q

import numpy as np
import pandas as pd
#import pycaret
import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import BertTokenizer, BertForMaskedLM
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
# specify GPU
device = torch.device("cuda")

# Set Working Directory - if working on Google Drive
# Mount Google Drive - applicable, if working on Google Drive
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive #/Colab_Notebooks


#**Load and prepare dataset(s) - generating 'final' versions**

In [ ]:
#==============================================================================================================================
#ISOT
#==============================================================================================================================

# Load dataset
true_data = pd.read_csv('/content/drive/MyDrive/DATASETS/ISOT_News_dataset/True.csv')
fake_data = pd.read_csv('/content/drive/MyDrive/DATASETS/ISOT_News_dataset/Fake.csv')

# Generate labels True/Fake under new Target Column in 'true_data' and 'fake_data'
true_data['label'] = ['True']*len(true_data)
fake_data['label'] = ['False']*len(fake_data)

# Merge 'true_data' and 'fake_data', by random mixing into a single df called 'data'
data = true_data._append(fake_data).sample(frac=1).reset_index().drop(columns=['index'])

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
data['label'] = pd.get_dummies(data.label)['True']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

#print(data)

df = pd.DataFrame(data)

#use title and text
df['text'] = df['title'] + '\n\n' + df['text']
finaldf = df[['text', 'label']]

#use only title
#df['text'] = df['title']
#finaldf = df[['text', 'label']]

#use only text
#finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final.csv", sep=',', index=False, encoding='utf-8')
#finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final_only_title.csv", sep=',', index=False, encoding='utf-8')
#finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/ISOT_final_only_text.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#NELA-GT-2018
#==============================================================================================================================

# Load dataset
from datasets import load_dataset
import sqlite3
#import create_engine from sqlalchemy
#from sqlalchemy import create_engine
#import pandas
import pandas as pd

#read database using create_engine
dbfile = '/content/drive/MyDrive/DATASETS/NELA-GT-2018/nela-gt-2018.db'
con = sqlite3.connect(dbfile)

# creating cursor
#cur = con.cursor()

# reading all table names
#table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
#print(table_list)

#takes about 5-6 mins due to large size of dataset
df = pd.read_sql_query("SELECT source, title, content FROM newsdata LIMIT 162411", con) #LIMIT to 20% of overall dataset (812 057)
#print(df)

# Be sure to close the connection
con.close()

df_labels = pd.read_csv("/content/drive/MyDrive/DATASETS/NELA-GT-2018/labels.csv", sep = ',', header=None)
df_labels = df_labels.drop(df_labels.index[0])
df_labels.columns = ['source', 'label']
#0 - reliable (TRUE), 1 - mixed (MIXED), 2 - unreliable (FALSE)
#df_labels = df_labels.drop(df_labels[df_labels['label'] == '1'].index)
df_labels.loc[df_labels.label == "0", 'label'] = "TRUE"
df_labels.loc[df_labels.label == "1", 'label'] = "MIXED"
df_labels.loc[df_labels.label == "2", 'label'] = "FALSE"
#print(df_labels)

df_merged = df.merge(df_labels, on = 'source', how = 'outer')
df_merged = df_merged.drop('source', axis=1)
df_merged = df_merged.drop(df_merged[df_merged['label'] == 'MIXED'].index)
df_merged = df_merged.rename(columns={"content": "text"})
#print(df_merged)

data = df_merged

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
data['label'] = pd.get_dummies(data.label)['TRUE']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

#print(data)

df = pd.DataFrame(data)

df['text'] = df['title'] + '\n\n' + df['text']
finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

#Check if None values exist in rows
#null_mask = finaldf.isnull().any(axis=1)
#null_rows = finaldf[null_mask]
#print(null_rows)

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2018_final.csv", sep=',', index=False, encoding='utf-8')

#Remove 'None' again
dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2018_final.csv")
dataset.set_format(type='pandas')
df = dataset['train'][:]
df = df[~df.isnull().any(axis=1)]
df.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2018_final.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#NELA-GT-2022
#==============================================================================================================================

# Load dataset
from datasets import load_dataset
import sqlite3
#import create_engine from sqlalchemy
#from sqlalchemy import create_engine
#import pandas
import pandas as pd

#read database using create_engine
dbfile = '/content/drive/MyDrive/DATASETS/NELA-GT-2022/nela-gt-2022.db'
con = sqlite3.connect(dbfile)

# creating cursor
#cur = con.cursor()

# reading all table names
#table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
#print(table_list)

#takes about 5-6 mins due to large size of dataset
df = pd.read_sql_query("SELECT source, title, content FROM newsdata LIMIT 177866", con) #LIMIT to 10% of overall dataset (1 778 361)
#print(df)

# Be sure to close the connection
con.close()

df_labels = pd.read_csv("/content/drive/MyDrive/DATASETS/NELA-GT-2022/labels.csv", sep = ',', header=None)
df_labels = df_labels.drop(df_labels.index[0])
df_labels.columns = ['source', 'label']
#0 - reliable (TRUE), 1 - mixed (MIXED), 2 - unreliable (FALSE)
#df_labels = df_labels.drop(df_labels[df_labels['label'] == '1'].index)
df_labels.loc[df_labels.label == "0", 'label'] = "TRUE"
df_labels.loc[df_labels.label == "1", 'label'] = "MIXED"
df_labels.loc[df_labels.label == "2", 'label'] = "FALSE"
#print(df_labels)

df_merged = df.merge(df_labels, on = 'source', how = 'outer')
df_merged = df_merged.drop('source', axis=1)
df_merged = df_merged.drop(df_merged[df_merged['label'] == 'MIXED'].index)
df_merged = df_merged.rename(columns={"content": "text"})
#print(df_merged)

data = df_merged

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
data['label'] = pd.get_dummies(data.label)['TRUE']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

#print(data)

df = pd.DataFrame(data)

df['text'] = df['title'] + '\n\n' + df['text']
finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

#Check if None values exist in rows
#null_mask = finaldf.isnull().any(axis=1)
#null_rows = finaldf[null_mask]
#print(null_rows)

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2022_final.csv", sep=',', index=False, encoding='utf-8')

#Remove 'None' again
dataset = load_dataset("csv", header=0, data_files="/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2022_final.csv")
dataset.set_format(type='pandas')
df = dataset['train'][:]
df = df[~df.isnull().any(axis=1)]
df.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/NELA-GT-2022_final.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#BuzzFeed - Store as CSV
#==============================================================================================================================

#Load dataset
#data = pd.read_xml('/content/drive/MyDrive/DATASETS/BuzzFeed-WebisFakeNewsCorpus2016/articls/articles')

import os
import pandas as pd
import xml.etree.ElementTree as et
df_cols= ['mainText', 'veracity']
df= pd.DataFrame([], columns=df_cols)
path='/content/drive/MyDrive/DATASETS/BuzzFeed-WebisFakeNewsCorpus2016/articles/articles/'

i = 0
df_cols= ['title','text','label']
df_final = pd.DataFrame([], columns=df_cols)

for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    xtree = et.parse(fullname)
    xroot = xtree.getroot()
    rows = []
    for record in xroot:
      mtitle = xroot.find('title').text
      mtext = xroot.find('mainText').text
      vlabel = xroot.find('veracity').text
      rows.append({"title": mtitle,"text": mtext,"label": vlabel})
    df_temp = pd.DataFrame(rows, columns = df_cols)
    df_final = pd.concat([df_final,df_temp])
    #df.to_csv('/content/drive/MyDrive/DATASETS/BuzzFeed-WebisFakeNewsCorpus2016/articles/articles_csv/'+ str(i) + '.csv', index=False)
    #i+=1

print(df_final)
df_final = df_final.drop_duplicates(subset=['text'])
print(df_final)
df_final.to_csv('/content/drive/MyDrive/DATASETS/BuzzFeed-WebisFakeNewsCorpus2016/BuzzFeed-WebisFakeNewsCorpus2016.csv', index=False)

In [ ]:
#==============================================================================================================================
#BuzzFeed
#==============================================================================================================================

data = pd.read_csv("/content/drive/MyDrive/DATASETS/BuzzFeed-WebisFakeNewsCorpus2016/BuzzFeed-WebisFakeNewsCorpus2016.csv", sep = ',', header=None)
data = data.drop(data.index[0])
data.columns = ['title', 'text', 'label']

#changing mostly false and no factual content labels to FALSE
data.loc[data.label == "mostly false", 'label'] = "FALSE"
data.loc[data.label == "no factual content", 'label'] = "FALSE"
#delete column with mixture of true and false (potentially this could be good data for a category 'mixed')
data = data.drop(data[data['label'] == 'mixture of true and false'].index)
#changing mostly-true label to TRUE
data.loc[data.label == "mostly true", 'label'] = "TRUE"

#print(data)

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
data['label'] = pd.get_dummies(data.label)['TRUE']  #dtype=int results in 0/1 not True/False

#print(data)

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
#print(label_size)
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

df = pd.DataFrame(data)

#Change capitalized titles to sentence cased (save as different csv)
df['title'] = df['title'].str.capitalize()

#merge title and text
df['text'] = df['title'] + '\n\n' + df['text']
#only text
#df['text'] = df['text']
finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

#finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_only_text.csv", sep=',', index=False, encoding='utf-8')
#finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final.csv", sep=',', index=False, encoding='utf-8')
finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/BuzzFeed-2016_final_titles_sentence_case.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#LIAR
#==============================================================================================================================
#Read dataset (as dataframe)
test_dataset = pd.read_csv("/content/drive/MyDrive/DATASETS/liar_dataset/test.tsv", sep = '\t', header=None)
train_dataset = pd.read_csv("/content/drive/MyDrive/DATASETS/liar_dataset/train.tsv", sep = '\t', header=None)
validate_dataset = pd.read_csv("/content/drive/MyDrive/DATASETS/liar_dataset/valid.tsv", sep = '\t', header=None)
#Merge datasets to one
data = test_dataset._append(train_dataset).sample(frac=1).reset_index().drop(columns=['index'])
data = data._append(validate_dataset).sample(frac=1).reset_index().drop(columns=['index'])
#add labels
data.columns = ['ID','label', 'text', 'subject', 'speaker', 'speaker job title', 'state info', 'party affiliation', 'barely true counts', 'false counts', 'half true counts', 'mostly true counts', 'pants on fire counts', 'context']
#print(data)                  #instead of 'statement' use label 'text'

#changing false and pants-fire labels to FALSE
data.loc[data.label == "pants-fire", 'label'] = "FALSE"
data.loc[data.label == "false", 'label'] = "FALSE"
#delete column with half-true or barely-true (potentially this could be good data for a category 'mixed')
#data.loc[data.label == "half-true", 'label'] = "FALSE"
#data.loc[data.label == "barely-true", 'label'] = "FALSE"
data = data.drop(data[data['label'] == 'half-true'].index)
data = data.drop(data[data['label'] == 'barely-true'].index)
#changing true and mostly-true labels to TRUE
data.loc[data.label == "true", 'label'] = "TRUE"
data.loc[data.label == "mostly-true", 'label'] = "TRUE"

#data.rename(columns = {'label':'target'}, inplace = True)

#random mixing of dataframe rows
#data = data. sample(frac=1)
#print(data)

# Label column is made of string values True/False, let's change it to numbers 0/1 (Real or TRUE=1)
data['label'] = pd.get_dummies(data.label)['TRUE']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

df = pd.DataFrame(data)
finaldf = df[["text", "label"]]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/LIAR_final.csv", sep=',', index=False, encoding='utf-8')

print(data)

In [ ]:
#==============================================================================================================================
#WELFAKE
#==============================================================================================================================

data = pd.read_csv("/content/drive/MyDrive/DATASETS/WELFake/WELFake_Dataset.csv", sep = ',')
print(data)

#(0 = fake and 1 = real)
data.loc[data.label == 1, 'label'] = "TRUE"
data.loc[data.label == 0, 'label'] = "FALSE"

# Label column is made of string values True/False, let's change it to numbers 0/1 (Real or TRUE=1)
data['label'] = pd.get_dummies(data.label)['TRUE']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

df = pd.DataFrame(data)

df['text'] = df['title'] + '\n\n' + df['text']
finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

finaldf = finaldf.drop_duplicates()

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/WELFake_final.csv", sep=',', index=False, encoding='utf-8')

print(finaldf)

In [ ]:
#==============================================================================================================================
#clmentbisaillon - fake-and-real-news-dataset / ISOT
#==============================================================================================================================

# Load dataset
true_data = pd.read_csv('/content/drive/MyDrive/DATASETS/clmentbisaillon - fake-and-real-news-dataset - nlp fake news detection dataset/True.csv')
fake_data = pd.read_csv('/content/drive/MyDrive/DATASETS/clmentbisaillon - fake-and-real-news-dataset - nlp fake news detection dataset/Fake.csv')

# Generate labels True/Fake under new Target Column in 'true_data' and 'fake_data'
true_data['label'] = ['True']*len(true_data)
fake_data['label'] = ['False']*len(fake_data)

# Merge 'true_data' and 'fake_data', by random mixing into a single df called 'data'
data = true_data._append(fake_data).sample(frac=1).reset_index().drop(columns=['index'])

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
data['label'] = pd.get_dummies(data.label)['True']

# Checking if our data is well balanced
label_size = [data['label'].sum(),len(data['label'])-data['label'].sum()]
plt.pie(label_size,explode=[0.1,0.1],colors=['navy','firebrick'],startangle=90,shadow=True,labels=['True','False'],autopct='%1.1f%%')
plt.show()

#print(data)

df = pd.DataFrame(data)

df['text'] = df['title'] + '\n\n' + df['text']
finaldf = df[['text', 'label']]

finaldf = finaldf.sample(frac=1, random_state=42).reset_index(drop=True)

finaldf.dropna(subset=['text'], inplace=False)
finaldf.dropna(subset=['label'], inplace=False)
# Select Rows without NaN Values in Specific Column
#finaldf = finaldf[~finaldf['text'].isna()]
# Select Rows without NaN Values in All Columns
finaldf = finaldf[~finaldf.isnull().any(axis=1)]

finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/clmentbisaillon_final.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#Fighting an Infodemic: COVID-19 Fake News Dataset
#==============================================================================================================================

#login to hugging face
!huggingface-cli login --token #hugging face token #--add-to-git-credential

from transformers import pipeline

from datasets import load_dataset

ds = load_dataset("nanyy1025/covid_fake_news")

print(ds)

ds = ds.rename_column("tweet", "text")
#label = string real vs fake
#adjust to true=1 and false=0

finaldf = ds['train'].to_pandas()

finaldf.loc[finaldf.label == "real", 'label'] = "TRUE"
finaldf.loc[finaldf.label == "fake", 'label'] = "FALSE"

# Label column is made of string values True/Fake, let's change it to numbers 0/1 (TRUE=1)
finaldf['label'] = pd.get_dummies(finaldf.label)['TRUE']  #dtype=int results in 0/1 not True/False

print(finaldf)
#finaldf.sample(n=100)

#ds['train'].to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',', index=False, encoding='utf-8')
finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/COVID19-fake-news_final.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
#==============================================================================================================================
#jy46604790 - fake dataset with EU news // EU vs Disinfo data
#==============================================================================================================================

#login to hugging face
!huggingface-cli login --token #hugging face token #--add-to-git-credential

from transformers import pipeline

from datasets import load_dataset

ds = load_dataset("winterForestStump/fake-news-detector-euvsdisinfo")

#print(ds)

ds = ds.rename_column("Title", "title")
ds = ds.rename_column("Summary", "text")

ds = ds['train'].add_column(name="label", column=[i for i in range(len(ds['train']))])

finaldf = ds.to_pandas()

finaldf = finaldf.assign(label=0) #Real or True = 1, False = 0 (all False for this dataset, no True available)

#print(finaldf)
#finaldf.sample(n=100)

#ds['train'].to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',', index=False, encoding='utf-8')
finaldf.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',', index=False, encoding='utf-8')

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#Inspect data
df = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep = ',')

#print all rows to inspect
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #print(df)

print(df)

#print("Rows:", len(df.index))

**Clean europeannewsroom data**

In [ ]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_dump.csv", sep = ';', header=None, names=['Date', 'Category', 'Title', 'Shorttext', 'Longtext'])
#print(df)

df.drop(df.columns[[0]], axis=1, inplace=True) #drop Date
df.drop(df.columns[[0]], axis=1, inplace=True) #drop Category
#print(df)

#Remove html from content/Longtext
for index, row in df.iterrows():
    row['Longtext'] = BeautifulSoup(str(row['Longtext'])).get_text()

df['Longtext'] = df['Shorttext'] + df['Longtext']
df.drop(df.columns[[1]], axis=1, inplace=True) #drop Shorttext

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (for this dataset all true so 1)

df.rename(columns={'Title': 'title', 'Longtext': 'text'}, inplace=True)

#display(df)

#save enr_dump_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final_uncleaned.csv", sep = ',', index = False)

#if necessary manually adjust any remaining coding errors or html and save as cleaned version = enr_final

**Select and clean EMNAD data**

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (for this dataset all true so 1)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample_final.csv", sep = ',', index = False)

**Each MS:**

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Belgium_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (all False for this dataset, no True available)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Belgium_sample_final.csv", sep = ',', index = False)

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Cyprus_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (all False for this dataset, no True available)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Cyprus_sample_final.csv", sep = ',', index = False)

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Ireland_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (all False for this dataset, no True available)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Ireland_sample_final.csv", sep = ',', index = False)

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Malta_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (all False for this dataset, no True available)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Malta_sample_final.csv", sep = ',', index = False)

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/United_Kingdom_sample.csv", sep = ',')

df.rename(columns = {'Article Body':'text'}, inplace = True)
df.rename(columns = {'Article Title':'title'}, inplace = True)
df = df[['text', 'title']]
df = df.reindex(columns=['title', 'text'])

#add True label
df.insert(2, "label", 1)  #Real or True = 1, False = 0 (all False for this dataset, no True available)

#display(df)

#save EMNAD_sample_final
df.to_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/United_Kingdom_sample_final.csv", sep = ',', index = False)

**Build final merged Datasets (enr_final.csv, EMNAD_sample_final.csv, and jy46604790_fake_dataset_final.csv)**

**Sizes:**

***enr_final.csv:*** 536 (uncleaned) - 510 (cleaned = final)

***EMNAD_sample_final.csv:*** 50,000

***jy46604790_fake_dataset_final.csv:*** 9,965

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df_EMNAD = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample_final.csv", sep = ',')
print(df_EMNAD)

In [ ]:
#==============================================================================================================================
#EU mix 1 ([9,455 EMNAD articles and 510 europeannewsroom articles] + 9,965 EUvsDisinfo articles)
#==============================================================================================================================
df_enr = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final.csv", sep = ',')
df_EMNAD = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_enr = df_enr.sample(n=510, random_state=42)
df_EMNAD = df_EMNAD.sample(n=9455, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=9965, random_state=42)

#merge dfs
df_mix = pd.concat([df_enr, df_EMNAD], ignore_index=True)
df_mix = pd.concat([df_mix, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_1_final.csv", sep=',', index = False)

In [ ]:
#==============================================================================================================================
#EU mix 2 ([510 EMNAD articles and 510 europeannewsroom articles] + 1,020 EUvsDisinfo articles)
#==============================================================================================================================
df_enr = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final.csv", sep = ',')
df_EMNAD = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_enr = df_enr.sample(n=510, random_state=42)
df_EMNAD = df_EMNAD.sample(n=510, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=1020, random_state=42)

#merge dfs
df_mix = pd.concat([df_enr, df_EMNAD], ignore_index=True)
df_mix = pd.concat([df_mix, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_2_final.csv", sep=',', index = False)

In [ ]:
#==============================================================================================================================
#EU mix 3 EMNAD (9,965 EMNAD articles + 9,965 EUvsDisinfo articles)
#==============================================================================================================================
df_EMNAD = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/EMNAD_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_EMNAD = df_EMNAD.sample(n=9965, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=9965, random_state=42)

#merge dfs
df_mix = pd.concat([df_EMNAD, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_3_EMNAD_final.csv", sep=',', index = False)

In [ ]:
#==============================================================================================================================
#EU mix 4 enr (510 europeannewsroom articles + 510 EUvsDisinfo articles)
#==============================================================================================================================
df_enr = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_enr = df_enr.sample(n=510, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=510, random_state=42)

#merge dfs
df_mix = pd.concat([df_enr, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_4_enr_final.csv", sep=',', index = False)

In [ ]:
#==============================================================================================================================
#EU mix 5 enr & Belgium (510 europeannewsroom articles + 9,455 Belgium articles + 9,965 EUvsDisinfo articles)
#==============================================================================================================================
df_enr = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final.csv", sep = ',')
df_Belgium = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Belgium_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_enr = df_enr.sample(n=510, random_state=42)
df_Belgium = df_Belgium.sample(n=9455, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=9965, random_state=42)

#merge dfs
df_mix = pd.concat([df_enr, df_Belgium], ignore_index=True)
df_mix = pd.concat([df_mix, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_5_enr_Belgium_final.csv", sep=',', index = False)

In [ ]:
#==============================================================================================================================
#EU mix 6 enr & Belgium (510 europeannewsroom articles + 510 Belgium articles + 1,020 EUvsDisinfo articles)
#==============================================================================================================================
df_enr = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_dpa_europeannewsroom_com/enr_final.csv", sep = ',')
df_Belgium = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Belgium_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_enr = df_enr.sample(n=510, random_state=42)
df_Belgium = df_Belgium.sample(n=510, random_state=42)
df_EUvsDis = df_EUvsDis.sample(n=1020, random_state=42)

#merge dfs
df_mix = pd.concat([df_enr, df_Belgium], ignore_index=True)
df_mix = pd.concat([df_mix, df_EUvsDis], ignore_index=True)

#merge title and text
df_mix['text'] = df_mix['title'] + '\n\n' + df_mix['text']

#save
df_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_6_enr_Belgium_final.csv", sep=',', index = False)

**Each MS:**

In [ ]:
#==============================================================================================================================
#EU MS (Belgium, Cyprus, Ireland, Malta, United Kingdom) + EUvsDisinfo (each 9,965:9,965 EUvsDisinfo, mix with same shuffle sample of EUvsDisinfo - enables a better comparison)
#==============================================================================================================================
df_Belgium = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Belgium_sample_final.csv", sep = ',')
df_Cyprus = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Cyprus_sample_final.csv", sep = ',')
df_Ireland = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Ireland_sample_final.csv", sep = ',')
df_Malta = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/Malta_sample_final.csv", sep = ',')
df_United_Kingdom = pd.read_csv("/content/drive/MyDrive/DATASETS/EU_European Multilingual News Articles Dataset with Topic Annotation/United_Kingdom_sample_final.csv", sep = ',')
df_EUvsDis = pd.read_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/jy46604790_fake_dataset_EU_final.csv", sep=',')

#random selection of portions
df_Belgium = df_Belgium.sample(n=9965, random_state=42)
df_Cyprus = df_Cyprus.sample(n=9965, random_state=42)
df_Ireland = df_Ireland.sample(n=9965, random_state=42)
df_Malta = df_Malta.sample(n=9965, random_state=42)
df_United_Kingdom = df_United_Kingdom.sample(n=9965, random_state=42)

df_EUvsDis = df_EUvsDis.sample(n=9965, random_state=42)

#merge dfs
df_Belgium_mix = pd.concat([df_Belgium, df_EUvsDis], ignore_index=True)
df_Cyprus_mix = pd.concat([df_Cyprus, df_EUvsDis], ignore_index=True)
df_Ireland_mix = pd.concat([df_Ireland, df_EUvsDis], ignore_index=True)
df_Malta_mix = pd.concat([df_Malta, df_EUvsDis], ignore_index=True)
df_United_Kingdom_mix = pd.concat([df_United_Kingdom, df_EUvsDis], ignore_index=True)

#merge title and text
df_Belgium_mix['text'] = df_Belgium_mix['title'] + '\n\n' + df_Belgium_mix['text']
df_Cyprus_mix['text'] = df_Cyprus_mix['title'] + '\n\n' + df_Cyprus_mix['text']
df_Ireland_mix['text'] = df_Ireland_mix['title'] + '\n\n' + df_Ireland_mix['text']
df_Malta_mix['text'] = df_Malta_mix['title'] + '\n\n' + df_Malta_mix['text']
df_United_Kingdom_mix['text'] = df_United_Kingdom_mix['title'] + '\n\n' + df_United_Kingdom_mix['text']

#save
df_Belgium_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Belgium_EUvsDis_final.csv", sep=',', index = False)
df_Cyprus_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Cyprus_EUvsDis_final.csv", sep=',', index = False)
df_Ireland_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Ireland_EUvsDis_final.csv", sep=',', index = False)
df_Malta_mix.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_Malta_EUvsDis_final.csv", sep=',', index = False)
df_United_Kingdom.to_csv("/content/drive/MyDrive/DATASETS/FINAL_DATASETS/EU_FINAL_DATASETS/EU_mix_United_Kingdom_EUvsDis_final.csv", sep=',', index = False)